# PROGRAMA PARA CALCULAR LAS MÉTRICAS A PARTIR DE LOS JSON GENERADOS

In [6]:
import json
import os
import datetime 

In [7]:
dirNames = ['Login', 'Logout', 'LoginZone', 'LogoutZone', 'OpenChest', 'OpenInv', 'OpenShop', 'OpenSkills', 'TutoTask', 'UsePot', 'UseSkill']

### Comprobación de los eventos

In [24]:
def getEventsFromPlayer(player):
    initial_count = 0
    
    dict_JSONS = {
        "Login": [],
        "Logout": [],
        "LoginZone": [],
        "LogoutZone": [],
        "OpenChest": [],
        "OpenInv": [],
        "OpenShop": [],
        "OpenSkills": [],
        "TutoTask": [],
        "UsePot": [],
        "UseSkill": [],
    }
    
    for directory in dirNames:
        if os.path.exists(directory + "/" + player) and os.path.isdir(directory + "/" + player):
            if not os.listdir(directory + "/" + player):
                print(f"No hay eventos de {directory} en el {player}")
            else:    
                print(f"Hay eventos de {directory} en el {player}.")
                for path in os.listdir(directory + "/" + player):
                    if os.path.isfile(os.path.join(directory + "/" + player, path)):
                        f = open(directory + "/" + player + "/" + directory + str(initial_count) + ".json", "r")
                        content = f.read()
                        jsondecoded = json.loads(content)
                        dict_JSONS[directory].insert(initial_count, jsondecoded)
                        initial_count += 1
            print(initial_count)
            initial_count = 0
        else:
            print(f"El directorio {directory} del {player} no existe.")
    return dict_JSONS

## VARIABLES GLOBALES

In [50]:
numPlayers = 3

dict_JSONS0 = getEventsFromPlayer("Player0")
dict_JSONS1 = getEventsFromPlayer("Player1")
dict_JSONS2 = getEventsFromPlayer("Player2")

Hay eventos de Login en el Player0.
3
Hay eventos de Logout en el Player0.
3
Hay eventos de LoginZone en el Player0.
13
Hay eventos de LogoutZone en el Player0.
12
Hay eventos de OpenChest en el Player0.
29
Hay eventos de OpenInv en el Player0.
34
Hay eventos de OpenShop en el Player0.
6
Hay eventos de OpenSkills en el Player0.
6
Hay eventos de TutoTask en el Player0.
5
El directorio UsePot del Player0 no existe.
Hay eventos de UseSkill en el Player0.
56
Hay eventos de Login en el Player1.
1
Hay eventos de Logout en el Player1.
1
Hay eventos de LoginZone en el Player1.
3
No hay eventos de LogoutZone en el Player1
0
Hay eventos de OpenChest en el Player1.
4
El directorio OpenInv del Player1 no existe.
El directorio OpenShop del Player1 no existe.
El directorio OpenSkills del Player1 no existe.
Hay eventos de TutoTask en el Player1.
5
El directorio UsePot del Player1 no existe.
El directorio UseSkill del Player1 no existe.
No hay eventos de Login en el Player2
0
No hay eventos de Logout 

## Método para hacer una media

In [10]:
def getMedia(params):
    sumatorio = 0;
    for i in params:
        sumatorio += i
    return sumatorio / params.length

## Método para calcular un porcentaje

In [11]:
def getPorcentaje(param, total):
    return (param / total) * 100

## Método para calcular el número de muertes del jugador pasado por parámetro.

In [20]:
def getNumDeaths(dict_JSON):
    cont_muertes = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == -1:
                    cont_muertes += 1
                else:
                    return cont_muertes
    return 0

### Realizamos el cálculo de la métrica: Media de número de muertes antes de completar la primera zona por primera vez (se compara entre las personas que han hecho el tutorial completo y las que no)

In [51]:
# Llamamos al método que nos devuelve los tiempos del tutorial para saber quienes han hecho el tutorial y quien no
array0 = getTimesTutorial(dict_JSONS0)
array1 = getTimesTutorial(dict_JSONS1)
array2 = getTimesTutorial(dict_JSONS2)

mediaSiTuto = 0
mediaNoTuto = 0
if len(array0) == 5:
    mediaSiTuto += getNumDeaths(dict_JSONS0)
else:
    mediaNoTuto += getNumDeaths(dict_JSONS0)
if len(array1) == 5:
    mediaSiTuto += getNumDeaths(dict_JSONS1)
else:
    mediaNoTuto += getNumDeaths(dict_JSONS1)
if len(array2)== 5:
    mediaSiTuto += getNumDeaths(dict_JSONS2)
else:
    mediaNoTuto += getNumDeaths(dict_JSONS2)

mediaSiTuto = mediaSiTuto / numPlayers
mediaNoTuto = mediaNoTuto / numPlayers

print("\nLa media de muertes de los jugadores que han hecho el tutorial es de:", mediaSiTuto)
print("La media de muertes de los jugadores que NO han hecho el tutorial es de:", mediaNoTuto)


La media de muertes de los jugadores que han hecho el tutorial es de: 0.0
La media de muertes de los jugadores que NO han hecho el tutorial es de: 0.0


## Método para calcular tiempo que tarda un jugador en pasar la primera zona

In [35]:
def getTimeZone(dict_JSON):
    timeIni = 0
    timeOut = 0
    result = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            date_timeIni = datetime.datetime.fromtimestamp(timeIni)
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    result = date_timeOut - date_timeIni
                    return result
    return datetime.timedelta(seconds=0)

### Realizamos el cálculo de la métrica: Media de tiempo que tardan los jugadores en superar la primera primera zona del juego (se compara entre las personas que han hecho el tutorial y las que no)

In [55]:
mediaSiTuto = 0
mediaNoTuto = 0

cont_jugadores_tuto = 0

if len(array0) == 5:
    mediaSiTuto += getTimeZone(dict_JSONS0).total_seconds()
    cont_jugadores_tuto += 1
else:
    mediaNoTuto += getTimeZone(dict_JSONS0).total_seconds()
if len(array1) == 5:
    mediaSiTuto += getTimeZone(dict_JSONS1).total_seconds()
    cont_jugadores_tuto += 1
else:
    mediaNoTuto += getTimeZone(dict_JSONS1).total_seconds()
if len(array2)== 5:
    mediaSiTuto += getTimeZone(dict_JSONS2).total_seconds()
    cont_jugadores_tuto += 1
else:
    mediaNoTuto += getTimeZone(dict_JSONS2).total_seconds()
    
mediaSiTuto = mediaSiTuto / cont_jugadores_tuto
mediaNoTuto = mediaNoTuto / (numPlayers - cont_jugadores_tuto)

print("\nLa media de tiempo 'segundos' que tardan los jugadores en superar la primera zona, si han hecho el tutorial es de:", mediaSiTuto)
print("La media de tiempo 'segundos' que tardan los jugadores en superar la primera zona, si NO han hecho el tutorial es de:", mediaNoTuto)


La media de tiempo 'segundos' que tardan los jugadores en superar la primera zona, si han hecho el tutorial es de: 52.5
La media de tiempo 'segundos' que tardan los jugadores en superar la primera zona, si NO han hecho el tutorial es de: 0.0


## Método para calcular el número de veces que se abre el inventario durante la primera zona por cada jugador

In [107]:
def getNumOpenInv(dict_JSON):
    cont_open = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"]== 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    for openInv in dict_JSON["OpenInv"]:
                        timeOpenInv=openInv["timestamp"]
                        if timeOpenInv > timeIni and timeOpenInv < timeOut :
                            cont_open+=1
    return cont_open

## Método para calcular si el jugador ha abierto el inventario poco después de abrir un cofre

In [103]:
def getNumOpenInvAfterChest(dict_JSON,timeToOpenChest):
    for openChest in dict_JSON["OpenChest"]:
            timeIni = openChest["timestamp"]
            for openInv in dict_JSON["OpenInv"]:
                    timeOut = openInv["timestamp"]
                    date_timeIni = datetime.datetime.fromtimestamp(timeIni)
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    if date_timeOut - date_timeIni < timeToOpenChest :
                        return True 

## Método para calcular si un jugador ha abierto el menú de habilidades antes de pasarse la primera zona

In [101]:
def getOpenSkill(dict_JSON):
    cont_open = 0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            date_timeIni = datetime.datetime.fromtimestamp(timeIni)
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    for openSkill in dict_JSON["OpenSkill"]:
                        timeOpen = openSkill["timestamp"]
                        date_timeOpen = datetime.datetime.fromtimestamp(timeOpen)
                        if date_timeOpen < date_timeOut and date_timeOpen > date_timeIni :
                            return True
                    return False

## Método para calcular si un jugador ha abierto la tienda la primera vez que vuelve al barco tras pasarse la primera zona

In [ ]:
def getOpenShopAfterCompleteZone(dict_JSON):
    for zone in dict_JSON["LogoutZone"]:
        if zone["zone"] == 11 and zone["next"] == 0:
            timeIni = zone["timestamp"
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["next"] == 0 :
                    timeOut = zoneOut["timestamp"]
                    for openShop in dict_JSON["OpenShop"]:
                        if openShop["timestamp"] < timeOut and openShop["timestamp"] > timeIni:
                            return True

## Método para calcular si un jugador ha usado habilidades al pasarse la primera zona

In [108]:
def getUseSkill(dict_JSON):
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 and not zone["completed"]:
            timeIni = zone["timestamp"]
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    date_timeIni = datetime.datetime.fromtimestamp(timeIni) 
                    date_timeOut = datetime.datetime.fromtimestamp(timeOut)
                    for useSkill in dict_JSON["UseSkill"]:
                        timeUse = useSkill["timestamp"]
                        date_timeUse = datetime.datetime.fromtimestamp(timeUse)
                        if date_timeUse < date_timeOut and date_timeUse > date_timeIni :
                            return True
                    return False

## Método para calcular el número de veces que un jugador se equipa una poción durante la primera zona

In [109]:
def getEquipSkill(dict_JSON):
    cont_equip=0
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11:
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeIni = zone["timestamp"]
                    timeOut = zoneOut["timestamp"]
                    for equipSkill in dict_JSON["OpenInv"]:
                        timeEquip = equipSkill["timestamp"]
                        if timeEquip < timeOut and timeEquip > timeIni and equipSkill["equipPot"] :
                            cont_equip+=1
    return cont_equip

## Método para calcular si un jugador usa una poción durante la primera zona

In [110]:
def getUsePot(dict_JSON):
    for zone in dict_JSON["LoginZone"]:
        if zone["zone"] == 11 :
            for zoneOut in dict_JSON["LogoutZone"]:
                if zoneOut["zone"] == 11 and zoneOut["next"] > -1:
                    timeOut = zoneOut["timestamp"]
                    timeIni = zone["timestamp"]
                    for usePot in dict_JSON["UsePot"]:
                        timeUse = usePot["timestamp"]
                        if timeUse < timeOut and timeUse > timeIni :
                            return True
                    return False 

## Método para calcular el tiempo que tarda en cada parte del tutorial cada jugador

In [46]:
def getTimesTutorial(dict_JSON):
    times = []
    i = 0
    for tuto in dict_JSON["TutoTask"]:
        times.insert(i, tuto["timeOut"])
        i += 1
    return times

## Método para calcular si el jugador se ha saltado el tutorial en algún punto

In [2]:
def getTutorialOut(dict_JSON):
    if dict_JSON["TutoTask"].length == 0 or dict_JSON["TutoTask"] < 5:
        return True
    else:
        return False